# Note Detection Console

Console template created on May 23 2014, class taken from the SciPy 2015 Vispy talk opening example <br>
see https://github.com/vispy/vispy/pull/928 https://github.com/flothesof/LiveFFTPitchTracker @author: florian <br>

librosa: https://librosa.github.io/librosa/generated/librosa.feature.chroma_cqt.html#librosa.feature.chroma_cqt <br>


## HYPERPARAMETERS in the program 
(not necessarily easily changed)
* chunksize
elaborated in class MicrophoneRecorder
* noise (for energy) 
determines the onset detection desensitivity
* threshold crossing point 
determines the onset detection sensitivity
* lower threshold of spectrum 
muting everything below the frequency

## ISSUES:
* lost frames, especially after updating graph 
I have no idea how to make the plotting happen on a separate thread

* harmonic problem:
lower frequencies: identifies the second harmonic as ffreq <br>
higher frequencies: identifies half of ffreq as ffreq <br>
no note may be identified if the drum and piano is hit at the same time <br>

## TO DO LIST:
* easier on-off switches
* display every frame
* exclude inharmonic sounds
* subtracting the spectrum before the onset
* multiplying the signal to analyse with a window
* changing some terminologies: "shift" to "delay"
* add some weights, adapt spread adapt spread based on how high ffreq is
* implement new pitch (identify region) precise pitch (specify point) HPS to improve the alogrithm.

Import packages

In [ ]:
# -*- coding: utf-8 -*-
import sys
import threading
import atexit
import pyaudio
import numpy as np
import matplotlib.pyplot as plt
import time
from PyQt4 import QtGui, uic, QtCore
from matplotlib.backends.backend_qt4agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.backends.backend_qt4agg import NavigationToolbar2QT as NavigationToolbar

For interfacing with the microphone

In [ ]:
class MicrophoneRecorder(object):
    """
    This microphone runs on an independent thread.
    It groups the signal in blocks of 2048 (chunksize) entries, as "frame"
    It accumulate these frames until "get_frames" is called, when it will pass over these entries.
    (There should be no accumulation of entries, else information is lost.)
    Choice of chunksize
    - large enough to determine the exact frequency
    - small enough to be responsive: to indicate the new note as promptly as possible
    """
    def __init__(self, rate=44100, chunksize=2048):
        the_input_device_index = 1  # to choose the microphone
        self.rate = rate  # sampling rate of microphone
        self.chunksize = chunksize  # size of each "frames"
        self.p = pyaudio.PyAudio()  # imported object to interface with the microphone
        self.stream = self.p.open(format=pyaudio.paInt16,  # sound take the format of int16
                                  channels=1,  # takes mono?
                                  rate=self.rate,  # sampling rate
                                  input=True,
                                  input_device_index=the_input_device_index,  # to choose the microphone
                                  frames_per_buffer=self.chunksize,  # size of each "frame"
                                  stream_callback=self.new_frame)  # function to call per "frame" generated
        print self.p.get_device_info_by_index(the_input_device_index)["name"]  # print mic name
        
        self.lock = threading.Lock()  # something to do with threading
        self.stop = False
        self.frames = []  # initiatlize frames
        atexit.register(self.close)

    def new_frame(self, data, frame_count, time_info, status):
        """
        function to call per "frame" generated
        each frame has "data"
        """
        data = np.fromstring(data, 'int16')
        with self.lock:  # using threading?
            self.frames.append(data)  # add data to the array of "frames"
            if self.stop:
                return None, pyaudio.paComplete
        return None, pyaudio.paContinue

    def get_frames(self):
        with self.lock:  # using threading?
            frames = self.frames  # return the frames accumulated - should have only one
            self.frames = []  # clear frames
            return frames

    def start(self):
        self.stream.start_stream()  # opening recording stream

    def close(self):  # some closing procedure, perhaps to erase memory
        with self.lock:
            self.stop = True
        self.stream.close()
        self.p.terminate()

For interfacing with the figure

In [ ]:
class MplFigure(object):  # don't know what is this for
    def __init__(self, parent):
        self.figure = plt.figure(facecolor='white')
        self.canvas = FigureCanvas(self.figure)
        self.toolbar = NavigationToolbar(self.canvas, parent)

In [ ]:
class LiveFFTWidget(QtGui.QWidget):
    def __init__(self):
        QtGui.QWidget.__init__(self)

        self.chunksize = 2048
        self.tempo_res = 32  # r_coeff resolution, needs to be a factor of chunksize
        self.tempo_num = int(self.chunksize/self.tempo_res)
        self.iteration = 0  # for counting, if needed
        self.noise = np.round(200000*np.random.randn(self.chunksize))  # to desensitise onset detection
        self.sampling_rate = 44100
        self.notes_dict = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]
        
        # holding variables, initialised with something random 
        
        self.signal_frame_pp0 = self.noise
        self.signal_frame_pp1 = self.noise
        self.signal_frame_pp2 = self.noise
        self.signal_frame_pp3 = self.noise
        
        self.energy_frame_pp0 = self.noise
        self.energy_frame_pp1 = self.noise
        self.energy_frame_pp2 = self.noise
        self.energy_frame_pp3 = self.noise
        
        self.rcoeff_frame_pp1 = [0.0] * int(self.tempo_res)
        self.rcoeff_frame_pp2 = [0.0] * int(self.tempo_res)
        self.rcoeff_frame_pp3 = [0.0] * int(self.tempo_res)
        
        self.bar_duration = 2.0  # how long your bar lasts
        self.bar_current = 0
        self.bar_cleared = 0
        self.loudness = 0
        self.bar_0_offsets = []
        self.bar_0_sizes = []
        self.bar_1_offsets = []
        self.bar_1_sizes = []
        self.bar_2_offsets = []
        self.bar_2_sizes = []
        self.bar_3_offsets = []
        self.bar_3_sizes = []
        
        self.note_detected = False
        self.note = "N.A."
        self.ffreq = 0.0
        self.signal_to_show = [0] * (self.chunksize*2)
        self.signal_to_ayse = [0] * (self.chunksize)
        self.shift = 0.0  # affects the part of the signal that will be analysed and plotted
        # customize the UI
        self.initUI()

        # init class data
        self.initData()

        # connect slots
        self.connectSlots()  # don't know what is this for

        # init MPL widget
        self.initMplWidget()  # (refer to MplFigure class)

        self.start_time = time.time()  # start timer
        self.prev_time = time.time()  # to calculate the time difference
#
        self.current_time = self.start_time

    def initUI(self):  # comment on this later
        hbox_gain = QtGui.QHBoxLayout()
        autoGain = QtGui.QLabel('Auto gain')
        autoGainCheckBox = QtGui.QCheckBox(checked=True)
        hbox_gain.addWidget(autoGain)
        hbox_gain.addWidget(autoGainCheckBox)

        # reference to checkbox
        self.autoGainCheckBox = autoGainCheckBox

        hbox_fixedGain = QtGui.QHBoxLayout()
        fixedGain = QtGui.QLabel('Fixed gain level')
        fixedGainSlider = QtGui.QSlider(QtCore.Qt.Horizontal)
        hbox_fixedGain.addWidget(fixedGain)
        hbox_fixedGain.addWidget(fixedGainSlider)

        self.fixedGainSlider = fixedGainSlider

        vbox = QtGui.QVBoxLayout()

        vbox.addLayout(hbox_gain)
        vbox.addLayout(hbox_fixedGain)

        # mpl figure
        self.main_figure = MplFigure(self)
        vbox.addWidget(self.main_figure.toolbar)
        vbox.addWidget(self.main_figure.canvas)

        self.setLayout(vbox)

        self.setGeometry(300, 300, 350, 300)
        self.setWindowTitle('LiveFFT')
        self.show()

        # timer for calls, taken from:
        # http://ralsina.me/weblog/posts/BB974.html
        timer = QtCore.QTimer()
        timer.timeout.connect(self.handleNewData)  # calls handleNewData every 20ms
        timer.start(10)  # chunks come out at a frequency of approximately 46ms
        # keep reference to timer
        self.timer = timer

    def initData(self):
        mic = MicrophoneRecorder(rate=44100, chunksize=self.chunksize)
        mic.start()

        # keeps reference to mic
        self.mic = mic

        # computes the parameters that will be used during plotting
        self.freq_vect = np.fft.rfftfreq(mic.chunksize, 1./mic.rate)  # original
        self.time_vect = np.arange(-mic.chunksize, mic.chunksize, dtype=np.float32) / mic.rate * 1000
        self.score_vect = np.arange(0, 4) #PCPS
        # the onset will be in the middle

    def connectSlots(self):
        pass  # don't know what is this for

    def initMplWidget(self):
        """
        creates initial matplotlib plots in the main window and keeps references for further use
        """
#         # top plot: currently to show energy
#         self.ax_top = self.main_figure.figure.add_subplot(211)
#         self.ax_top.set_ylim(-32768, 32768)  # original
#         # self.ax_top.set_ylim(-32768 * 100, 32768 * 100)  # to show energy
#         self.ax_top.set_xlim(self.time_vect.min(), self.time_vect.max())
#         self.ax_top.set_xlabel(u'time (ms)', fontsize=6)
        
        # top plot: now showing score and where you hit
        self.ax_top = self.main_figure.figure.add_subplot(211)
        self.ax_top.set_ylim(-3, 15)  # original
        self.ax_top.set_xlim(0, 4)
        self.ax_top.set_xlabel(u'bar number', fontsize=6)

        # bottom plot: currently to show spectrum
        self.ax_bottom = self.main_figure.figure.add_subplot(212)
        self.ax_bottom.set_ylim(0, 1)
        # self.ax_bottom.set_xlim(0, self.freq_vect.max()) original
        self.ax_bottom.set_xlim(0, 5000.)
        self.ax_bottom.set_xlabel(u'frequency (Hz)', fontsize=6)

        # line objects
#         self.line_top, = self.ax_top.plot(self.time_vect,
#                                           np.ones_like(self.time_vect), lw=0.2)
        
        self.bar_0 = self.ax_top.scatter([], [], s = [], edgecolors='none')
        self.bar_1 = self.ax_top.scatter([], [], s = [], edgecolors='none')
        self.bar_2 = self.ax_top.scatter([], [], s = [], edgecolors='none')
        self.bar_3 = self.ax_top.scatter([], [], s = [], edgecolors='none')
        
        self.line_bottom, = self.ax_bottom.plot(self.freq_vect,
                                                np.ones_like(self.freq_vect), lw=0.5)

        self.pitch_line, = self.ax_bottom.plot((self.freq_vect[self.freq_vect.size / 2],
                                                self.freq_vect[self.freq_vect.size / 2]),
                                               self.ax_bottom.get_ylim(), lw=2)
        # This plots for vertical line that marks the pitch
        # plt.tight_layout()  # tight layout

    def handleNewData(self):
        """ handles the asynchronously collected sound chunks """
#
        if self.iteration == 0:
            self.start_time = time.time()  # set to the true start time    
        
#         print str(time.time() - self.start_time) + "  " + str(time.time() - self.prev_time) + \
#         " gets the latest frames"
#         self.prev_time = time.time()
        signal_frames = self.mic.get_frames()

        # print str(time.time() - self.start_time) + "  " + str(time.time() - self.prev_time) + \
        # " taking last frame"
        # self.prev_time = time.time()
        if len(signal_frames) > 0:
            if len(signal_frames) > 1:
                print str(len(signal_frames) - 1) + " frame lost"
                # indicate number of frames lost - should not have any
            self.signal_frame_pp0 = signal_frames[-1]  # keeps only the last frame
            
            # print str(time.time() - self.start_time) + "  " + str(time.time() - self.prev_time) + \
            # " energy calculations"  # 0.01s
            # self.prev_time = time.time()
            # to calculate the rectangular window for every sample
            # numpy operations are more efficient than using python loops
            # the size of the rectangular window is one chunksize
            # convolution can be considered
            self.energy_frame_pp0 = np.full(self.chunksize, sum(np.absolute(self.signal_frame_pp2)), dtype="int32")
            to_cumsum = np.add(np.absolute(self.signal_frame_pp1), -np.absolute(self.signal_frame_pp2))
            cumsum = np.cumsum(to_cumsum)
            self.energy_frame_pp0[1:] = np.add(self.energy_frame_pp0[1:], cumsum[:-1])
            self.energy_frame_pp0 = np.add(self.energy_frame_pp0, self.noise)
            
            # print str(time.time() - self.start_time) + "  " + str(time.time() - self.prev_time) + \
            # " r_coeff calculations"
            # self.prev_time = time.time()
            # calculating pearson correlation coefficient at 2048/32 samples
            # to determine exact time of onset
            # could not think of any way this could be parallelised
            energy_arg = np.concatenate((self.energy_frame_pp1, self.energy_frame_pp0))
            # energy_arg = np.concatenate((self.energy_frame_pp1[i*self.tempo_num:],
            #                              self.energy_frame_pp0[:-(self.tempo_res-i)*self.tempo_num]))
            for i in range(self.tempo_res):
                self.rcoeff_frame_pp1[i] = np.corrcoef(energy_arg[i*self.tempo_num:(i*self.tempo_num+self.chunksize)],
                                                       np.arange(self.chunksize))[0, 1]            
            
#             print str(time.time() - self.start_time) + "  " + str(time.time() - self.prev_time) + \
#             " detecting new note"
#             self.prev_time = time.time()
            rcoeff_arg = np.concatenate((self.rcoeff_frame_pp2, self.rcoeff_frame_pp1))
            # we need the previous rcoeff frame to determine onset

            # finding the onset, any way not to loop?
#             for i in range(self.tempo_res, 0, -1):
            for _ in range(1):
#                 if rcoeff_arg[-i] > 0.80 and all(i < 0.80 for i in rcoeff_arg[-i-5:-i]):
                # if rcoeff_arg[-i] > 0.80 and np.max(rcoeff_arg[-i-31:-i]) < 0.80:
                if True:
                    # to determine onset  - where the rcoeff graph crosses 0.80,
                    # 31 entries cooldown - check that previous entries do not have cooldown
                    # print i
                    # print rcoeff_arg[-i]
                    # print np.around(rcoeff_arg, 2)

                    # print str(time.time() - self.start_time) + "  " + str(time.time() - self.prev_time) + \
                    # " note class"
                    # self.prev_time = time.time()
                    time_arg = np.concatenate((self.signal_frame_pp3, self.signal_frame_pp2,
                                               self.signal_frame_pp1, self.signal_frame_pp0))
                    self.signal_to_show = time_arg[-i*self.tempo_num - int((2+self.shift)*self.chunksize):
                                                   -i*self.tempo_num - int((0+self.shift)*self.chunksize)]
                    self.signal_to_ayse = time_arg[-i*self.tempo_num - int((1+self.shift)*self.chunksize):
                                                   -i*self.tempo_num - int((0+self.shift)*self.chunksize)]
                    signal_to_deduct = time_arg[-i*self.tempo_num - int((2+self.shift)*self.chunksize):
                                                -i*self.tempo_num - int((1+self.shift)*self.chunksize)]
                    # Consider whether should a window be applied

                    spectrum = np.absolute(np.fft.fft(self.signal_to_ayse))
                    spectrum_to_deduct = np.absolute(np.fft.fft(signal_to_deduct))
                    to_subtract = False  # take the spectral difference between the current and previous chunk
                    if to_subtract:
                        spectrum = np.clip(np.add(spectrum, -1 * np.array(spectrum_to_deduct)), 0, 100000000)
                        # consider the effectiveness of taking the difference

                    # self.spectrum = np.array(spectrum[:int(0.5*self.chunksize)+1])  # to be plotted MOVED TO LATER
                    # following is the hps algorithm
                    spectrum[:12] = 0.0  # anything below middle C is muted
#                     spectrum[:6] = 0.0  # anything below C3 is muted
                    spectrum[1024:] = 0.0  # mute second half of spectrum, lazy to change code

                    scale1 = [0.0] * (2048 * 6)
                    scale2 = [0.0] * (2048 * 6)
                    scale3 = [0.0] * (2048 * 6)

                    # upsampling the original scale spectrum, 6 for 1
                    scale1_f1 = np.convolve(spectrum, [5.0 / 6.0, 1.0 / 6.0])[1:]
                    scale1_f2 = np.convolve(spectrum, [4.0 / 6.0, 2.0 / 6.0])[1:]
                    scale1_f3 = np.convolve(spectrum, [3.0 / 6.0, 3.0 / 6.0])[1:]
                    scale1_f4 = np.convolve(spectrum, [2.0 / 6.0, 4.0 / 6.0])[1:]
                    scale1_f5 = np.convolve(spectrum, [1.0 / 6.0, 5.0 / 6.0])[1:]
                    scale1[::6] = spectrum
                    scale1[1::6] = scale1_f5
                    scale1[2::6] = scale1_f4
                    scale1[3::6] = scale1_f3
                    scale1[4::6] = scale1_f2
                    scale1[5::6] = scale1_f1
                    # downsampling from the 6 for 1 upsample
                    scale2[:2048 * 3] = scale1[::2]
                    scale3[:2048 * 2] = scale1[::3]
                    hps = np.prod((scale1, scale2, scale3), axis=0)  # the "product" in harmonic product spectrum
                    hps_max = np.argmax(hps)  # determine the location of the peak of hps result
                    # calculate the corresponding frequency of the peak
                    self.ffreq = hps_max * 44100.0 / (2048.0 * 6.0)  # sampling rate / (chunksize * upsampling value)

                    self.spectrum = np.array(spectrum[:int(0.5*self.chunksize)+1])  # to be plotted MOVED FROM EARILER

                    if hps_max < 5:
                        print "low ffreq"  # should not be possible - just investigating
                        break

                    # TODO: add some weights, adapt spread based on how high ffreq is
                    total_energy = np.sum(scale1)
                    total_energy_due_to_ffreq = np.sum(scale1[::hps_max]) \
                                                + np.sum(scale1[1::hps_max]) + np.sum(scale1[:hps_max - 1:hps_max]) \
                                                # + np.sum(scale1[2::hps_max]) + np.sum(scale1[:hps_max - 2:hps_max]) \
                                                # + np.sum(scale1[3::hps_max]) + np.sum(scale1[:hps_max - 3:hps_max]) \
                                                # + np.sum(scale1[4::hps_max]) + np.sum(scale1[:hps_max - 4:hps_max]) \
                                                # + np.sum(scale1[5::hps_max]) + np.sum(scale1[:hps_max - 5:hps_max]) \
                                                # + np.sum(scale1[6::hps_max]) + np.sum(scale1[:hps_max - 6:hps_max])
#
                    self.loudness = total_energy_due_to_ffreq  # exported for plotting the bar chart
                                            
                    portion_of_energy = (total_energy_due_to_ffreq/total_energy)*20

#                     if portion_of_energy > 1:
                    if True:
                        # printing note in solfage form
                        note_no = -3 + (np.log2(self.ffreq) - np.log2(220.0)) * 12.0  # take logarithm and find note
#                         note_no = -3 + (np.log2(self.ffreq) - np.log2(110.0)) * 12.0  # take logarithm and find note
                        note_no_rounded = np.round(note_no)  # round off to nearest note
                        note_no_difference = note_no - note_no_rounded
                        octave_no = 4 + int(note_no_rounded // 12)
#                         octave_no = 3 + int(note_no_rounded // 12)
                        solfate_no = int(note_no_rounded) % 12
                        self.note = str(self.notes_dict[solfate_no]) + str(octave_no)
                        
                        self.note_no = note_no  # exported for later usage
                        
                        print ("{:.2f}Hz({:02}) {:.2f}, {:3s} {:+.2f} at {:.3f}s"
                               .format(self.ffreq, int(note_no_rounded), portion_of_energy, self.note, note_no_difference,
                                       time.time() - self.start_time))
                        self.note_detected = True
                    else:
#                         print("inharmonic sound ({:.2f}) detected at {:.3f}s"
#                               .format(portion_of_energy, time.time() - self.start_time))
                        pass

            display_only_note = False
            if self.note_detected or not display_only_note and self.iteration > 0:  # first loop have ambiguous loudness
#                 print str(time.time() - self.start_time) + "  " + str(time.time() - self.prev_time) + \
#                 " set time on graph"  # 0.001s
#                 self.prev_time = time.time()

# 
#                 self.line_top.set_data(self.time_vect, self.signal_to_show)  # plots the time signal, onset on middle
                # self.line_top.set_data(self.time_vect, np.concatenate((self.signal_frame_pp2, self.signal_frame_pp1)))
                # self.line_top.set_data(self.time_vect, energy_arg)  # plots the energy
#                 self.ax_top.scatter(2 + np.random.randn(), 2 + np.random.randn(), [10])
#                 self.bar_1 = self.ax_top.scatter([2 + np.random.randn()], [2 + np.random.randn()], [10])
#                 self.bar_1.set_offsets([[2 + np.random.random(), 2 + np.random.random()], 
#                                         [2 + np.random.random(), 2 + np.random.random()]])
#                 self.bar_1.set_sizes([2 + 20*np.random.random(), 2 + 20*np.random.random()])


                self.loudness = int(1+self.loudness/100000.)  # need a better function lulz
#                 print self.loudness

                self.current_time = time.time() - self.start_time                
                self.bar_current = np.floor(self.current_time / self.bar_duration)
                residual_bar = (self.current_time - (self.bar_current * self.bar_duration))/self.bar_duration
#                 print residual_time
        
                if self.bar_current - self.bar_cleared > 2:
                    bar_to_clear = (self.bar_current - 3) % 4
                    self.bar_cleared = self.bar_current - 2
                    if bar_to_clear == 0:
                        self.bar_0_offsets = []
                        self.bar_0_sizes = []
                        self.bar_0.set_offsets([])
                        self.bar_0.set_sizes([])
                    elif bar_to_clear == 1:
                        self.bar_1_offsets = []
                        self.bar_1_sizes = []
                        self.bar_1.set_offsets([])
                        self.bar_1.set_sizes([])
                    elif bar_to_clear == 2:
                        self.bar_2_offsets = []
                        self.bar_2_sizes = []
                        self.bar_2.set_offsets([])
                        self.bar_2.set_sizes([])
                    elif bar_to_clear == 3:
                        self.bar_3_offsets = []
                        self.bar_3_sizes = []
                        self.bar_3.set_offsets([])
                        self.bar_3.set_sizes([])
                
                if self.bar_current%4 == 0:
                    self.bar_0_offsets.append([0 + residual_bar, self.note_no])
                    self.bar_0_sizes.append(self.loudness)
                    self.bar_0.set_offsets(self.bar_0_offsets)
                    self.bar_0.set_sizes(self.bar_0_sizes)
                elif self.bar_current%4 == 1:
                    self.bar_1_offsets.append([1 + residual_bar, self.note_no])
                    self.bar_1_sizes.append(self.loudness)
                    self.bar_1.set_offsets(self.bar_1_offsets)
                    self.bar_1.set_sizes(self.bar_1_sizes)
                elif self.bar_current%4 == 2:
                    self.bar_2_offsets.append([2 + residual_bar, self.note_no])
                    self.bar_2_sizes.append(self.loudness)
                    self.bar_2.set_offsets(self.bar_2_offsets)
                    self.bar_2.set_sizes(self.bar_2_sizes)
                elif self.bar_current%4 == 3:
                    self.bar_3_offsets.append([3 + residual_bar, self.note_no])
                    self.bar_3_sizes.append(self.loudness)
                    self.bar_3.set_offsets(self.bar_3_offsets)
                    self.bar_3.set_sizes(self.bar_3_sizes) 
                  

                # print str(time.time() - self.start_time) + "  " + str(time.time() - self.prev_time) + \
                # " take FFT"
                # self.prev_time = time.time()
                # fft_frame = self.spectrum
                # fft_frame = np.fft.rfft(self.signal_to_ayse)  # computes and plots the fft signal
                # print len(fft_frame)
                # print len(self.spectrum)
                fft_frame = self.spectrum  # 1025 entries

                # print str(time.time() - self.start_time) + "  " + str(time.time() - self.prev_time) + \
                # " some thing about scaling"
                # self.prev_time = time.time()
                # inherited, don't know what is this for
                # perhaps it is to normalise the spectrum - plotting is faster without changing axes
                if self.autoGainCheckBox.checkState() == QtCore.Qt.Checked:
                    fft_frame /= np.abs(fft_frame).max()
                else:
                    fft_frame *= (1 + self.fixedGainSlider.value()) / 5000000.
                    # print(np.abs(fft_frame).max())

                # print str(time.time() - self.start_time) + "  " + str(time.time() - self.prev_time) + \
                # " set spectrum on graph"  # 0.001 s
                # self.prev_time = time.time()
                self.line_bottom.set_data(self.freq_vect, np.abs(fft_frame))

                # print str(time.time() - self.start_time) + "  " + str(time.time() - self.prev_time) + \
                # " placeholder"  # 0.8s wdf
                # self.prev_time = time.time()

                new_pitch = self.ffreq
                precise_pitch = self.ffreq

                # print str(time.time() - self.start_time) + "  " + str(time.time() - self.prev_time) + \
                # " set pitch on graph"
                # self.prev_time = time.time()
                
                self.ax_bottom.set_title("pitch = {:.2f} Hz ({})".format(precise_pitch, self.note))
                self.pitch_line.set_data((new_pitch, new_pitch),
                                         self.ax_bottom.get_ylim())  # move the vertical pitch line

                if self.iteration % 1 == 0:  # update plot only after every n chunks, if necessary
                    print str(time.time() - self.start_time) + "  " + str(time.time() - self.prev_time) + \
                    " refresh plot"  # 0.105s
                    self.prev_time = time.time()
                    self.main_figure.canvas.draw()  # refreshes the plots, takes the bulk of time
                    print str(time.time() - self.start_time) + "  " + str(time.time() - self.prev_time) + \
                    " refreshed plot"  # 0.105s
                    self.prev_time = time.time()    

                self.note_detected = False

#             print str(time.time() - self.start_time) + "  " + str(time.time() - self.prev_time) + \
#             " storing for recursion"
#             self.prev_time = time.time()
            self.signal_frame_pp3 = self.signal_frame_pp2[:]
            self.signal_frame_pp2 = self.signal_frame_pp1[:]
            self.signal_frame_pp1 = self.signal_frame_pp0[:]
            self.energy_frame_pp1 = self.energy_frame_pp0[:]
            self.rcoeff_frame_pp2 = self.rcoeff_frame_pp1[:]

        self.iteration += 1
        #print str(time.time() - self.start_time) + "  " + str(time.time() - self.prev_time) + \
        # " end of loop \n \n"
        # self.prev_time = time.time()

## Execute Code

In [ ]:
if __name__ == "__main__":
    app = QtGui.QApplication(sys.argv)
    window = LiveFFTWidget()
    sys.exit(app.exec_())